# 🎓 Day 1-00.05: LoRA 파인튜닝 개념 이해하기 (초보자용)

## 🎯 이번 노트북에서 할 일
- **LoRA의 핵심 개념** 이해하기
- **파인튜닝 과정** 시각화로 보기
- **실제 코드** 맛보기 (실행하지 않음)
- **메모리 절약** 원리 이해하기

## 💡 LoRA란 무엇인가?

### 🔧 LoRA (Low-Rank Adaptation)
**전체 모델을 다시 학습하지 않고, 작은 어댑터만 추가해서 학습하는 방법**

### 🎯 왜 LoRA를 사용할까?

#### ❌ 일반 파인튜닝의 문제점
- **메모리 부족**: 7B 모델 = 28GB+ 메모리 필요
- **시간 오래 걸림**: 전체 모델을 다시 학습
- **비용 높음**: GPU 리소스 많이 사용
- **과적합 위험**: 작은 데이터로 큰 모델 학습

#### ✅ LoRA의 장점
- **메모리 절약**: 1%만 학습 (28GB → 2GB)
- **빠른 학습**: 작은 어댑터만 업데이트
- **안정적**: 원본 모델 성능 유지
- **효율적**: 여러 태스크에 재사용 가능

### 🧠 LoRA 작동 원리

#### 1️⃣ 기존 방식 (Full Fine-tuning)
```
원본 모델 (7B 파라미터)
    ↓ 전체 학습
새로운 모델 (7B 파라미터)
```

#### 2️⃣ LoRA 방식
```
원본 모델 (7B 파라미터) + LoRA 어댑터 (0.1B 파라미터)
    ↓ 어댑터만 학습
원본 모델 + 학습된 어댑터
```

### 🔍 LoRA 어댑터 구조
```
입력 → [원본 레이어] → [LoRA A] → [LoRA B] → 출력
           (고정)        (학습)     (학습)
```

- **원본 레이어**: 학습하지 않음 (고정)
- **LoRA A, B**: 작은 행렬들만 학습
- **결과**: 원본 + 어댑터의 조합


## 1. LoRA 파라미터 이해하기


In [ ]:
# LoRA의 핵심 파라미터들을 이해해봅시다
print("🔧 LoRA 파라미터 설명")
print("=" * 50)

# 1. r (rank) - LoRA의 핵심 파라미터
print("1️⃣ r (rank): LoRA의 '크기'")
print("   - 작을수록: 빠르고 메모리 적게 사용, 정확도 낮음")
print("   - 클수록: 느리고 메모리 많이 사용, 정확도 높음")
print("   - 일반적 범위: 4, 8, 16, 32, 64")
print("   - 예시: r=16 → 16차원 어댑터")

print("\n2️⃣ lora_alpha: 학습 강도 조절")
print("   - 높을수록: 더 강하게 학습")
print("   - 낮을수록: 더 부드럽게 학습")
print("   - 일반적 값: r의 2배 (r=16 → alpha=32)")

print("\n3️⃣ target_modules: 어떤 레이어를 학습할지")
print("   - 'q_proj': Query 프로젝션 (어텐션의 질문 부분)")
print("   - 'v_proj': Value 프로젝션 (어텐션의 값 부분)")
print("   - 'k_proj': Key 프로젝션 (어텐션의 키 부분)")
print("   - 'o_proj': Output 프로젝션 (어텐션 출력)")

print("\n4️⃣ lora_dropout: 과적합 방지")
print("   - 0.05~0.1: 일반적 범위")
print("   - 높을수록: 과적합 방지, 학습 어려움")
print("   - 낮을수록: 학습 쉬움, 과적합 위험")

# 실제 LoRA 설정 예시
print("\n📋 실제 LoRA 설정 예시:")
print("   r=16                    # 적당한 크기")
print("   lora_alpha=32           # r의 2배")
print("   target_modules=['q_proj', 'v_proj']  # 어텐션 레이어들")
print("   lora_dropout=0.05       # 과적합 방지")


## 2. 메모리 절약 효과 시각화


In [ ]:
# LoRA의 메모리 절약 효과를 시각화해봅시다
import matplotlib.pyplot as plt
import numpy as np

print("📊 LoRA 메모리 절약 효과 비교")
print("=" * 50)

# 모델 크기별 메모리 사용량 (GB)
models = ['7B 모델', '13B 모델', '30B 모델', '70B 모델']
full_tuning = [28, 52, 120, 280]  # Full fine-tuning 메모리
lora_tuning = [2, 4, 8, 16]       # LoRA 메모리

# 그래프 생성
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# 1. 메모리 사용량 비교
x = np.arange(len(models))
width = 0.35

bars1 = ax1.bar(x - width/2, full_tuning, width, label='Full Fine-tuning', color='red', alpha=0.7)
bars2 = ax1.bar(x + width/2, lora_tuning, width, label='LoRA', color='green', alpha=0.7)

ax1.set_xlabel('모델 크기')
ax1.set_ylabel('메모리 사용량 (GB)')
ax1.set_title('메모리 사용량 비교')
ax1.set_xticks(x)
ax1.set_xticklabels(models)
ax1.legend()
ax1.grid(True, alpha=0.3)

# 값 표시
for bar in bars1:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 1,
             f'{int(height)}GB', ha='center', va='bottom')

for bar in bars2:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.5,
             f'{int(height)}GB', ha='center', va='bottom')

# 2. 절약률 계산
savings = [(f-l)/f*100 for f, l in zip(full_tuning, lora_tuning)]
bars3 = ax2.bar(models, savings, color='blue', alpha=0.7)

ax2.set_xlabel('모델 크기')
ax2.set_ylabel('메모리 절약률 (%)')
ax2.set_title('LoRA 메모리 절약률')
ax2.grid(True, alpha=0.3)

# 값 표시
for bar in bars3:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 1,
             f'{height:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# 절약 효과 요약
print("\n💡 LoRA 메모리 절약 효과:")
for i, model in enumerate(models):
    print(f"   {model}: {full_tuning[i]}GB → {lora_tuning[i]}GB ({savings[i]:.1f}% 절약)")

print(f"\n🎯 핵심 포인트:")
print(f"   - 7B 모델: 28GB → 2GB (93% 절약!)")
print(f"   - 70B 모델: 280GB → 16GB (94% 절약!)")
print(f"   - 일반적인 GPU로도 대형 모델 파인튜닝 가능")


## 3. 실제 파인튜닝 코드 맛보기 (실행하지 않음)


In [ ]:
# 실제 파인튜닝 코드를 맛보기로 살펴봅시다 (실행하지 않음!)
print("📝 실제 파인튜닝 코드 구조")
print("=" * 50)

print("1️⃣ 데이터 준비 단계:")
print("""
# RAFT 데이터 로드
raft_dataset = load_from_disk("data/raft_dataset")

# 토큰화 함수
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding=True,
        max_length=512
    )

# 데이터셋 토큰화
tokenized_dataset = raft_dataset.map(tokenize_function, batched=True)
""")

print("\n2️⃣ 학습 설정 단계:")
print("""
# 학습 설정
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,           # 3번 반복 학습
    per_device_train_batch_size=4, # 배치 크기
    gradient_accumulation_steps=4, # 그래디언트 누적
    warmup_steps=100,             # 워밍업
    learning_rate=2e-4,           # 학습률
    fp16=True,                    # 16비트 학습
    logging_steps=10,             # 로깅 주기
    save_steps=500,               # 저장 주기
)
""")

print("\n3️⃣ 학습 실행 단계:")
print("""
# 데이터 정리기
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # 언어 모델링
)

# 트레이너 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

# 실제 학습 실행 (이 부분이 시간이 오래 걸림!)
print("🚀 파인튜닝 시작...")
trainer.train()
""")

print("\n4️⃣ 모델 저장 단계:")
print("""
# 학습된 모델 저장
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

print("✅ 파인튜닝 완료!")
""")

print("\n💡 실제 실행 시 주의사항:")
print("   - GPU 메모리: 최소 8GB 이상 필요")
print("   - 학습 시간: 데이터 크기에 따라 1-10시간")
print("   - 모니터링: 학습 과정을 실시간으로 확인")
print("   - 저장: 정기적으로 체크포인트 저장")


## 4. LoRA vs Full Fine-tuning 비교


In [ ]:
# LoRA와 Full Fine-tuning을 비교해봅시다
print("⚖️ LoRA vs Full Fine-tuning 비교")
print("=" * 50)

# 비교표 생성
import pandas as pd

comparison_data = {
    '항목': [
        '메모리 사용량',
        '학습 시간',
        '모델 크기',
        '학습 파라미터',
        '정확도',
        '안정성',
        '비용',
        '재사용성'
    ],
    'Full Fine-tuning': [
        '28GB+ (7B 모델)',
        '매우 오래 (10+ 시간)',
        '7B 파라미터',
        '7B 파라미터',
        '매우 높음',
        '낮음 (과적합 위험)',
        '매우 높음',
        '어려움'
    ],
    'LoRA': [
        '2GB (7B 모델)',
        '빠름 (1-3 시간)',
        '7B + 0.1B 어댑터',
        '0.1B 파라미터',
        '높음 (95% 수준)',
        '높음 (원본 유지)',
        '낮음',
        '쉬움 (어댑터만 교체)'
    ]
}

df = pd.DataFrame(comparison_data)
print(df.to_string(index=False))

print("\n🎯 핵심 차이점:")
print("1️⃣ 메모리: LoRA가 93% 절약")
print("2️⃣ 시간: LoRA가 3-5배 빠름")
print("3️⃣ 정확도: LoRA가 95% 수준 유지")
print("4️⃣ 비용: LoRA가 10배 저렴")

print("\n💡 언제 무엇을 사용할까?")
print("✅ LoRA 사용 시기:")
print("   - 메모리가 부족할 때")
print("   - 빠른 실험이 필요할 때")
print("   - 여러 태스크를 시도할 때")
print("   - 비용을 절약하고 싶을 때")

print("\n✅ Full Fine-tuning 사용 시기:")
print("   - 최고 정확도가 필요할 때")
print("   - 충분한 메모리가 있을 때")
print("   - 한 번만 학습할 때")
print("   - 연구 목적일 때")

# 데이터셋 토큰화
print("📝 토큰화 실행 중...")
tokenized_dataset = raft_dataset.map(
    tokenize_function,
    batched=True,                   # 배치로 처리 (빠름)
    remove_columns=raft_dataset.column_names  # 원본 컬럼 제거
)

print("✅ 토큰화 완료!")
print(f"   - 토큰화된 데이터 개수: {len(tokenized_dataset):,}개")
print(f"   - 토큰 길이: {tokenized_dataset[0]['input_ids'].shape[1]}개")

# 샘플 확인
print(f"\n🔍 토큰화 결과 샘플:")
sample_tokens = tokenized_dataset[0]['input_ids'][:20]  # 처음 20개 토큰
sample_text = tokenizer.decode(sample_tokens, skip_special_tokens=True)
print(f"   - 토큰 ID: {sample_tokens.tolist()}")
print(f"   - 디코딩된 텍스트: {sample_text[:100]}...")


## 5. 다음 단계 안내 (매우 간단하게!)


In [ ]:
# 다음 단계 안내 및 요약
print("🎯 다음 단계 안내")
print("=" * 50)

print("📚 이제 다음 노트북들로 넘어가세요:")
print("1️⃣ 00.06-evaluation.ipynb: 모델 평가하기")
print("2️⃣ main-practice/03_fine_tuning_with_lora.ipynb: 실제 파인튜닝 실행")

print("\n💡 지금까지 배운 LoRA 핵심 개념:")
print("✅ LoRA란: 작은 어댑터만 추가해서 학습하는 방법")
print("✅ 메모리 절약: 93% 절약 (28GB → 2GB)")
print("✅ 시간 단축: 3-5배 빠름")
print("✅ 정확도: 95% 수준 유지")
print("✅ 비용 절약: 10배 저렴")

print("\n🔧 LoRA 핵심 파라미터:")
print("✅ r (rank): 16 (적당한 크기)")
print("✅ lora_alpha: 32 (r의 2배)")
print("✅ target_modules: 어텐션 레이어들")
print("✅ lora_dropout: 0.05 (과적합 방지)")

print("\n🎓 RAFT + LoRA 조합의 장점:")
print("✅ RAG 성능 향상: 문서 필터링 + 인용 정확도")
print("✅ 효율적 학습: LoRA로 빠르고 저렴하게")
print("✅ 도메인 특화: 특정 데이터에 특화된 성능")
print("✅ 실용적: 실제 서비스에 바로 적용 가능")

print("\n🚀 준비 완료!")
print("이제 실제 파인튜닝을 실행해보세요!")


## 6. 데이터 콜레이터 설정하기


In [ ]:
# 데이터 콜레이터를 설정합니다 (배치 데이터를 정리하는 도구)
print("📦 데이터 콜레이터 설정 중...")

# 데이터 콜레이터 생성
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,        # 토크나이저
    mlm=False,                  # 마스크 언어 모델링 사용 안함 (GPT 스타일)
    pad_to_multiple_of=8        # 8의 배수로 패딩 (GPU 효율성)
)

print("✅ 데이터 콜레이터 설정 완료!")
print("   - 언어 모델링: GPT 스타일 (다음 토큰 예측)")
print("   - 패딩: 8의 배수로 정렬")
print("   - 마스킹: 사용 안함")

# 데이터 콜레이터 테스트
print(f"\n🧪 데이터 콜레이터 테스트:")
test_batch = [tokenized_dataset[i] for i in range(2)]  # 2개 샘플로 테스트
collated = data_collator(test_batch)
print(f"   - 배치 크기: {collated['input_ids'].shape[0]}")
print(f"   - 시퀀스 길이: {collated['input_ids'].shape[1]}")
print(f"   - 라벨 크기: {collated['labels'].shape}")
print(f"   - 패딩 토큰 ID: {tokenizer.pad_token_id}")


## 7. 트레이너 생성하기


In [ ]:
# 트레이너를 생성합니다 (실제 학습을 실행하는 도구)
print("🏃 트레이너 생성 중...")

# 트레이너 생성
trainer = Trainer(
    model=model,                    # 학습할 모델
    args=training_args,             # 학습 설정
    train_dataset=tokenized_dataset, # 학습 데이터
    data_collator=data_collator,    # 데이터 콜레이터
    tokenizer=tokenizer             # 토크나이저
)

print("✅ 트레이너 생성 완료!")
print(f"   - 모델: LoRA 파인튜닝 모델")
print(f"   - 데이터: {len(tokenized_dataset):,}개 샘플")
print(f"   - 에포크: {training_args.num_train_epochs}회")
print(f"   - 총 스텝: {len(tokenized_dataset) * training_args.num_train_epochs // training_args.gradient_accumulation_steps:,}개")

# 학습 전 모델 테스트
print(f"\n🧪 학습 전 모델 테스트:")
test_prompt = "질문: 인공지능이란 무엇인가요?\n답변:"
inputs = tokenizer(test_prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=30,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"   - 입력: {test_prompt}")
print(f"   - 출력: {generated_text}")
print(f"   - 생성된 부분: {generated_text[len(test_prompt):]}")


## 8. 실제 파인튜닝 실행하기! 🚀


In [ ]:
# 드디어 실제 파인튜닝을 실행합니다!
print("🚀 파인튜닝 시작!")
print("=" * 50)
print("💡 이 과정은 시간이 걸릴 수 있습니다.")
print("💡 GPU를 사용하면 더 빠르게 학습됩니다.")
print("💡 학습 과정을 실시간으로 모니터링할 수 있습니다.")
print("=" * 50)

# 파인튜닝 실행
try:
    # 실제 학습 시작!
    trainer.train()
    
    print("\n🎉 파인튜닝 완료!")
    print("✅ 모델이 성공적으로 학습되었습니다!")
    
except Exception as e:
    print(f"\n❌ 파인튜닝 중 오류 발생: {e}")
    print("💡 GPU 메모리가 부족할 수 있습니다. 배치 크기를 줄여보세요.")
    print("💡 또는 Google Colab의 GPU를 사용해보세요.")


## 9. 학습된 모델 테스트하기


In [ ]:
# 학습된 모델이 얼마나 개선되었는지 테스트해봅시다!
print("🧪 학습된 모델 테스트 중...")

# 테스트 질문들
test_questions = [
    "질문: 인공지능이란 무엇인가요?\n답변:",
    "질문: 머신러닝과 딥러닝의 차이점은 무엇인가요?\n답변:",
    "질문: 자연어 처리는 무엇인가요?\n답변:"
]

print("📝 학습 전후 비교:")
print("=" * 60)

for i, question in enumerate(test_questions, 1):
    print(f"\n🔍 테스트 {i}: {question.split('질문: ')[1].split('\\n')[0]}")
    print("-" * 40)
    
    # 입력 토큰화
    inputs = tokenizer(question, return_tensors="pt").to(model.device)
    
    # 텍스트 생성
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,        # 최대 50개 토큰 생성
            temperature=0.7,          # 창의성 조절
            do_sample=True,           # 샘플링 사용
            pad_token_id=tokenizer.eos_token_id
        )
    
    # 생성된 텍스트 디코딩
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = generated_text[len(question):].strip()
    
    print(f"답변: {answer}")
    print(f"전체: {generated_text}")

print("\n✅ 모델 테스트 완료!")
print("💡 학습된 모델이 더 나은 답변을 생성하는지 확인해보세요!")


## 10. 학습된 모델 저장하기


In [ ]:
# 학습된 모델을 저장합니다 (나중에 사용할 수 있게!)
print("💾 학습된 모델 저장 중...")

# 1. LoRA 어댑터만 저장 (가벼움)
model.save_pretrained("models/fine_tuned_lora")

# 2. 토크나이저 저장
tokenizer.save_pretrained("models/fine_tuned_lora")

# 3. 학습 설정 저장
training_config = {
    "model_name": model_name,
    "lora_config": model_config['lora_config'],
    "training_args": {
        "num_train_epochs": training_args.num_train_epochs,
        "per_device_train_batch_size": training_args.per_device_train_batch_size,
        "learning_rate": training_args.learning_rate,
        "gradient_accumulation_steps": training_args.gradient_accumulation_steps,
        "warmup_steps": training_args.warmup_steps,
        "weight_decay": training_args.weight_decay
    },
    "dataset_info": {
        "total_samples": len(tokenized_dataset),
        "max_length": 512
    }
}

with open("models/training_config.json", "w", encoding="utf-8") as f:
    json.dump(training_config, f, ensure_ascii=False, indent=2)

print("✅ 모델 저장 완료!")
print("   - models/fine_tuned_lora/: LoRA 어댑터")
print("   - models/training_config.json: 학습 설정")
print("   - 다음 단계에서 이 모델을 사용할 수 있습니다!")

# 저장된 파일 확인
import os
print(f"\n📁 저장된 파일들:")
for root, dirs, files in os.walk("models/fine_tuned_lora"):
    for file in files:
        file_path = os.path.join(root, file)
        file_size = os.path.getsize(file_path) / (1024*1024)  # MB
        print(f"   - {file_path}: {file_size:.1f} MB")


## 11. 다음 단계 안내

### 🎯 다음 노트북에서 할 일
**00.06-evaluation.ipynb**에서:
1. **학습된 모델** 로드하기
2. **성능 평가**하기
3. **원본 모델과 비교**하기
4. **결과 분석**하기

### 💡 지금까지 배운 것
- ✅ RAFT 데이터 로드 및 토큰화
- ✅ LoRA 파인튜닝 설정
- ✅ 실제 파인튜닝 실행
- ✅ 학습 과정 모니터링
- ✅ 학습된 모델 저장

### 🔧 파인튜닝의 핵심
- **데이터**: RAFT 형식의 질문-답변 데이터
- **모델**: EXAONE + LoRA 어댑터
- **학습**: 3 에포크, 배치 크기 1, 그래디언트 누적 4
- **결과**: RAG 성능 향상된 모델

### 🚀 준비 완료!
이제 다음 노트북으로 넘어가서 학습된 모델의 성능을 평가해보겠습니다!
